In [3]:
pip install pyodbc

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pyodbc
import datetime
from datetime import datetime


In [2]:
import datetime 
from datetime import datetime  
import pyodbc   
class database_connectivity:  
    def __init__(self):  
        try: 
            self.conn = pyodbc.connect(        
                'Driver={SQL Server};'
                'Server=DESKTOP-25TJ8LJ\\SQLEXPRESS;'    
                'Database=menudb;'                       
                'Trusted_Connection=yes;')
            self.cursor = self.conn.cursor()  
            print("Database is connected properly...")
        except pyodbc.Error:
            print("Database connection Failed...")  
            print("Error: ",pyodbc.Error)

    def fetch_menu(self):  
        try:  
            self.cursor.execute("SELECT ItemID, ItemName, Price from Menu")       
            return self.cursor.fetchall()       
        except pyodbc.Error:
            print("Error while fetching menu:")
            return[]
        
    def fetch_item(self,item_name): 
        try:
            self.cursor.execute("SELECT ItemId,price from Menu where lower(ItemName)=lower(?)",(item_name,))  
            return self.cursor.fetchone()
        except pyodbc.Error:
            print("Error while fetching items: ",pyodbc.Error)
            return None
        
    def insert_customer_info(self,name,mobile): 
        try:
            self.cursor.execute("Insert INTO Customers(Name,mobile) values(?,?)",(name,mobile))  
            self.conn.commit()
            self.cursor.execute("Select @@IDENTITY")  
            return self.cursor.fetchone()[0]
        except pyodbc.Error:
            print("Inserting customer_info error",pyodbc.Error)
            return None
    
    def insert_order_info(self,customer_id,item_id,quantity,amount):
        try:
            self.cursor.execute("Insert into  Orders (CustomerID,ItemID,Quantity,Amount,Orderdate) values(?,?,?,?,getdate())",(customer_id,item_id,quantity,amount))
            self.conn.commit()
        except pyodbc.Error:
            print("inserting order_info error",pyodbc.Error)

    def close(self):
        try:
            self.conn.close()
            print("Database connected...")
        except:
            pass

In [3]:
class menu(): 
    def __init__(self,db):  
        self.db=db

    def display(self):     
        rows=self.db.fetch_menu()
        print("\n"+"-"*40)
        print("          Menu")
        print("-"*40)
        print("{:<5} {:<20} {:>10}".format("ID","Item Name","Price"))
        print("-"*40)
        for row in rows:
            print("{:<5} {:<20} {:>10}". format(row[0],row[1],row[2]))

In [4]:
class Order:  
    def __init__(self,db):
        self.db=db
        self.order_items=[] 
        self.total_amount=0  
    def take_order(self):
        while True:
            item_name=input("Enter Item name(or type 'done' to finish):")
            if item_name.lower()=='done':
                break
            try:
                quantity=int(input("Enter Quantity: "))
            except ValueError:
                print("Invalid quantity. Please enter a number.")
                continue
            result=self.db.fetch_item(item_name)
            if  result:
                item_id,price=result
                amount=price*quantity
                self.total_amount +=amount
                self.order_items.append({
                    "id":item_id,
                    "name":item_name,
                    "quantity":quantity,
                    "amount":amount})
            else:
                print("Item not found")
    def Print_bill(self,name,mobile):
        print("\n" + "=" * 45)
        print("Hotel Bill".center(45))
        print("=" * 45)
        print(f"\nCustomer Name:{name}")
        print(f"Mobile       :{mobile}")
        print(f"Date & Time  :{datetime.now().strftime('%d-%m-%Y %H:%M:%S')}")
        print("-"*45)
        print("{:<20} {:>10} {:>10}".format("Item", "Qty", "Amount"))
        print("-"*45)
        for  item in self.order_items:
            print("\n{:<20} {:>10} {:>10.2f}".format(
                item['name'],
                item['quantity'],
                item['amount']))
        print("-"*45)
        print("{:<5} {:<20} {:>10}".format("","Total:",self.total_amount)) 
        print("="*45)

    def save_order(self,customer_id):
        for item in self.order_items:
            self.db.insert_order_info(
                customer_id,
                item['id'],
                item['quantity'],
                item['amount']
            ) 

In [5]:
class Hotel_Bill_print:
    def __init__(self):
        self.db=database_connectivity()
        self.menu=menu(self.db)
        self.order=Order(self.db)
    def run(self):
        try:
            self.menu.display() 
            
            choice=input("\n Do you want to place order?(yes/no): ").lower() 
            if choice !='yes':
                print("Thank You...")
                return
            name=input("Enter name: ") 
            mobile=input("Enter mobile number: ")
            self.order.take_order() 
            self.order.Print_bill(name,mobile) 
            customer_id=self.db.insert_customer_info(name,mobile)
            if customer_id:  
                self.order.save_order(customer_id)  
        except Exception:
            print("Error: ",Exception)
        finally: 
            self.db.close()

In [6]:
obj= Hotel_Bill_print()
obj.run()

Database is connected properly...

----------------------------------------
          Menu
----------------------------------------
ID    Item Name                 Price
----------------------------------------
1     Vada Pav                  20.00
2     Upma                      30.00
3     Samosa                    15.00
4     Sandwich                  40.00
5     Pav Bhaji                 80.00
6     Masala Dosa               70.00
7     Plain Dosa                50.00
8     Cold Coffee               60.00
9     Lassi                     45.00
10    Pizza                    120.00
11    Burger                    90.00
12    French Fries              70.00
13    Paneer Tikka             150.00
14    Biryani                  180.00
15    Thali                    200.00

                  Hotel Bill                 

Customer Name:Vaishnavi
Mobile       :7465893746
Date & Time  :20-02-2026 10:45:14
---------------------------------------------
Item                        Qty     Amount